_Anmerkung: Ggf. müssen sie nach dem Öffnen des Notebooks noch einmal auf 'Kernel > Restart & Run All' clicken_

<h1>Abfragen mit SPARQL</h1>

In diesem Demonstrator werden wir zeigen, wie SPARQL Abfragen genutzt werden können, 
um aus Open Data Beständen weiteres Wissen zu ziehen.

![SPARQL Logo](./media/SPARQL.png)

Zuerst: Was ist SPARQL?
Bei SPARQL handelt es sich um die **S**PARQL **P**rotocol **A**nd **R**DF **Q**uery **L**anguage.
Es ist also sowohl ein Protokoll als auch eine Abfragesprache für das Resource Description Framework. 
Während SQL die Abfragesprache für relationale Datenbanken ist, ist SPARQL die geläufige Abfragesprache für RDF-Daten, welche nicht als Tabellen sondern als Graph mit gelabelten Kanten zu verstehen sind.

Eine SPARQL Abfrage ähnelt in ihren Augen möglicherweise einer SQL-Abfrage, jedoch sind die Gemeinsamkeiten nur oberflächlich.
Abfragen besitzen einen SELECT-Part, der die resultierenden Variablen (von der Struktur den Spalten in SQL entsprechend) wiedergibt und einem WHERE-Part, der definiert, welche Eigenschaften die resultierenden Filme alle gemeinsam besitzen.

In [1]:
from data.sparql import query, table

table(query("""
SELECT ?movie
WHERE
{
 ?movie wdt:P31 wd:Q11424
}
LIMIT 10
"""))

| movie |
| --- |
| http://www.wikidata.org/entity/Q372 |
| http://www.wikidata.org/entity/Q593 |
| http://www.wikidata.org/entity/Q595 |
| http://www.wikidata.org/entity/Q1365 |
| http://www.wikidata.org/entity/Q2201 |
| http://www.wikidata.org/entity/Q2345 |
| http://www.wikidata.org/entity/Q2875 |
| http://www.wikidata.org/entity/Q3092 |
| http://www.wikidata.org/entity/Q3187 |
| http://www.wikidata.org/entity/Q3208 |


'showing 10 results'

Diese Query liefert die ersten 10 Filme, welche in Wikidata gelistet sind.
Zwar können sie auf die Einträge klicken, um zu sehen, um was es sich handelt, jedoch ist dieses Format noch nicht sonderlich leserlich.

Um statt der Objekte selbst ihre Label zu erhalten, erweitern wir die vorherige Query. Diesmal soll nicht der Film selbst, sondern sein Titel angezeigt werden.

In [8]:
from data.sparql import query, table

table(query(
"""
SELECT ?title
WHERE
{
 ?movie wdt:P31 wd:Q11424 .
 ?movie wdt:P1448 ?title       
} 
LIMIT 10
"""
))    

| title |
| --- |
| ...men Olsenbanden var ikke død |
| Le guerriere dal seno nudo |
| La vendetta dei gladiatori |
| Zinebi Festival Internacional de Cine Documental y Cortometraje de Bilbao |
| Relatives |
| Börn |
| Syurpriz |
| Petualang Tak Kenal Menyerah |
| طلق صناعي |
| Brexit (2019) |


'showing 10 results'

Wie sie sehen, muss nicht jede Variable auch ausgegeben werden.
Hier sieht man nur den Titel, aber nicht das Film-Objekt.

Zudem ist dies die erste Query, bei der mehrere Kriterien (nämlich Objekt ?movie ist vom Typ Film und Object ?movie hat Titel ?title.
Jedes dieser logischen Tripletts wird durch einen Punkt vom nächsten getrennt.
Um etwas Schreibarbeit zu sparen, kann auch 

```
SELECT ?title
WHERE
{
 ?movie wdt:P31 wd:Q11424;
        wdt:P1448 ?title       
} 
LIMIT 10
```

schreiben. Hier wird durch das Semikolon ";" das Subjekt aus dem letzten Triplet für das Nächste wiederverwendet.

Schreiben sie ebenfalls eine Abfrage, welche die IMDB ID dreier Filme ausgibt.
Das Prädikat für 'hat IMDB ID' lautet 'wdt:P345', bei der IMDB ID handelt es sich um eine ID der 'Internet Movie Database'.

In [11]:
# Tragen sie hier ihren SPARQL-Code ein und klicken sie auf RUN, um ihn auszutesten.
# Denken sie daran, ggf. über 'from data.sparql import query, table' die notwendigen Funktionen zu importieren,
# sollte es sie hier zu einem Fehler kommen.








| title |
| --- |
| tt0498329 |
| tt1741542 |
| tt1675434 |
| tt0073817 |
| tt1250777 |
| tt0050083 |
| tt0031381 |
| tt0046031 |
| tt0062218 |
| tt0996979 |


'showing 10 results'

<h2> Anreicherung lokaler Daten mit Daten aus dem Netz </h2>

Die durch SPARQL verfügbar gemachten Daten können nun verwendet werden, 
um beispielsweise zusätzliche Informationen zu denen einer Datenbank hinzuzufügen.
In diesem Notebook handelt es sich hierbei um eine MongoDB, welche eine vielzahl an Filmen enthält.

In [20]:
import data.movies as movies

movies.get_by_title('Iron Man')

{'index': 12588,
 'adult': 'False',
 'belongs_to_collection': "{'id': 131292, 'name': 'Iron Man Collection', 'poster_path': '/fbeJ7f0aD4A112Bc1tnpzyn82xO.jpg', 'backdrop_path': '/rI8zOWkRQJdlAyQ6WJOSlYK6JxZ.jpg'}",
 'budget': '140000000',
 'genres': "[{'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 12, 'name': 'Adventure'}]",
 'homepage': 'http://www.ironmanmovie.com/',
 'id': '1726',
 'imdb_id': 'tt0371746',
 'original_language': 'en',
 'original_title': 'Iron Man',
 'overview': 'After being held captive in an Afghan cave, billionaire engineer Tony Stark creates a unique weaponized suit of armor to fight evil.',
 'popularity': '22.073099',
 'poster_path': '/848chlIWVT41VtAAgyh9bWymAYb.jpg',
 'production_companies': "[{'name': 'Paramount Pictures', 'id': 4}, {'name': 'Marvel Studios', 'id': 420}]",
 'production_countries': "[{'iso_3166_1': 'US', 'name': 'United States of America'}]",
 'release_date': '2008-04-30',
 'revenue': 585174222.0,
 'runtime': 126.0,

Um die in der Datenbank befindlichen Einträge zu Filmen um zusätzliche Informationen zu erweitern, muss zunächst ein geeigneter eindeutiger Identifier für einen Film gefunden werden. Dazu eignen sich einige Einträge eines Films besser als andere.

Frage: Welche Eigenschaft eines Filmes haltet ihr für sinvoll um online nach Infos zu suchen und warum?





<h3> Suche nach Namen: </h3>

In [34]:
from data.sparql import query, data, table, entity_to_json

#movie is instance (P31) of film (Q11424)
#movie has label ?titel

table(query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        rdfs:label ?title ;
  FILTER CONTAINS(?title, "Iron Man")
  FILTER LANGMATCHES(lang(?title),'en')
} 
"""
))


| movie |title |
| --- | --- |
| http://www.wikidata.org/entity/Q185884 |Tetsuo: The Iron Man |
| http://www.wikidata.org/entity/Q192724 |Iron Man |
| http://www.wikidata.org/entity/Q192724 |Iron Man |
| http://www.wikidata.org/entity/Q192724 |Iron Man |
| http://www.wikidata.org/entity/Q205028 |Iron Man 2 |
| http://www.wikidata.org/entity/Q205028 |Iron Man 2 |
| http://www.wikidata.org/entity/Q205028 |Iron Man 2 |
| http://www.wikidata.org/entity/Q209538 |Iron Man 3 |
| http://www.wikidata.org/entity/Q209538 |Iron Man 3 |
| http://www.wikidata.org/entity/Q209538 |Iron Man 3 |
| http://www.wikidata.org/entity/Q1758603 |Iron Man |
| http://www.wikidata.org/entity/Q1782442 |The Invincible Iron Man |
| http://www.wikidata.org/entity/Q1782442 |The Invincible Iron Man |
| http://www.wikidata.org/entity/Q1782442 |The Invincible Iron Man |
| http://www.wikidata.org/entity/Q3820040 |Iron Man |
| http://www.wikidata.org/entity/Q7742600 |The Iron Man |
| http://www.wikidata.org/entity/Q15971587 |Iron Man: Rise of Technovore |
| http://www.wikidata.org/entity/Q16831976 |Iron Man XXX |
| http://www.wikidata.org/entity/Q18150641 |Iron Man & Hulk: Heroes United |
| http://www.wikidata.org/entity/Q27039121 |Iron Man: The Road to Civil War |
| http://www.wikidata.org/entity/Q28677944 |Iron Man & Captain America: Heroes United |
| http://www.wikidata.org/entity/Q55648698 |The Iron Man |


'showing 22 results'

<h3> Suche nach imdb_id: </h3>

In [35]:
from data.sparql import query, table

table(query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        wdt:P345 ?imdb_id .
 FILTER CONTAINS (?imdb_id, "tt0371746")
} 
LIMIT 1
"""
))



| movie |imdb_id |
| --- | --- |
| http://www.wikidata.org/entity/Q192724 |tt0371746 |


'showing 1 results'

Über den Namen des Films oder andere Attribute wie die IMDB-ID können weitere Informationen gesucht werden:

In [48]:
from data.sparql import query, data, table, entity_to_json

result = query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        rdfs:label ?title ;
        wdt:P345 ?imdb_id .
  FILTER CONTAINS(?title, "Iron Man")
  FILTER LANGMATCHES(lang(?title),'en')
} 
LIMIT 1
"""
)

# a bit hacky, but it retrieves the id from the wikidata URI, which has the form https://www.wikidata.org/wiki/{id}
entity_id = data(result)[0]['movie'].split("/").pop()

entity_to_json(entity_id)

{'entities': {'Q185884': {'pageid': 184692,
   'ns': 0,
   'title': 'Q185884',
   'lastrevid': 1432440153,
   'modified': '2021-05-31T15:58:32Z',
   'type': 'item',
   'id': 'Q185884',
   'labels': {'de': {'language': 'de', 'value': 'Tetsuo: The Iron Man'},
    'en': {'language': 'en', 'value': 'Tetsuo: The Iron Man'},
    'es': {'language': 'es', 'value': 'Tetsuo: The Iron Man'},
    'fr': {'language': 'fr', 'value': 'Tetsuo'},
    'it': {'language': 'it', 'value': 'Tetsuo'},
    'ja': {'language': 'ja', 'value': '鉄男'},
    'lv': {'language': 'lv', 'value': 'Tecuo: Dzelzs cilvēks'},
    'pl': {'language': 'pl', 'value': 'Tetsuo: The Iron Man'},
    'ru': {'language': 'ru', 'value': 'Тэцуо — железный человек'},
    'nl': {'language': 'nl', 'value': 'Tetsuo: The Iron Man'},
    'eml': {'language': 'eml', 'value': 'Tetsuo'},
    'fa': {'language': 'fa', 'value': 'تتسوئو: مرد آهنین'},
    'pt': {'language': 'pt', 'value': 'Tetsuo'},
    'zh': {'language': 'zh', 'value': '鐵男'},
    'uk': {

Hier erkennt man, dass wir eine große Menge an Daten zum Film erhalten. 
Wir können hier den Film-Titel in vielen Sprachen angeben, herausfinden, welcher Direktor für den Schnitt des Filmes verantwortlich ist, welche unterschiedlichen Jugendschutz-Label ein Film hat, etc.

<h3> Einfügen aller Schauspieler eines Films </h3>

In [3]:
from data.sparql import query, data, table, entity_to_json

result = query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        rdfs:label ?title ;
        wdt:P345 ?imdb_id .
  FILTER CONTAINS(?title, "Iron Man")
  FILTER LANGMATCHES(lang(?title),'en')
} 
LIMIT 1
"""
)

Einige der Eigenschaften sind jedoch über ihre Wikidata-ID aufgelistet. Diese muss aufgelöst werden, um menschenverständliche Daten zu erhalten, also z.B. das es sich bei 'P345' um eine 'IMDB ID' handelt. 

In [2]:
from data.sparql import query, data, table, entity_to_json

result = query(
"""
SELECT ?titleLabel ?castLabel ?property ?propLabel
WHERE {
    ?title wdt:P345 "tt0371746".
    # take all claims on this movie
    ?title ?property ?cast .
    # that involve a human
    ?cast wdt:P31 wd:Q5 .

    # get the property label
    # see https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries#Adding_labels_for_properties
    hint:Query hint:optimizer "None" .
    ?prop wikibase:directClaim ?property .

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
)

table (result)

| titleLabel |castLabel |property |propLabel |
| --- | --- | --- | --- |
| The Avengers |Seamus McGarvey |http://www.wikidata.org/prop/direct/P344 |director of photography |
| The Avengers |Paul Bettany |http://www.wikidata.org/prop/direct/P725 |voice actor |
| The Avengers |Lou Ferrigno |http://www.wikidata.org/prop/direct/P725 |voice actor |
| The Avengers |Jeffrey Ford |http://www.wikidata.org/prop/direct/P1040 |film editor |
| The Avengers |Stan Lee |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| The Avengers |Jon Favreau |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| The Avengers |Alexandra Byrne |http://www.wikidata.org/prop/direct/P2515 |costume designer |
| The Avengers |Jeff Wolfe |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Rashmi Rustagi |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Katsumi Komatsu |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Yumiko Komatsu |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Momoko Komatsu |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Fernanda Toker |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Andrea Vecchio |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Michael Zhang |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |William Christopher Stephens |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Kelley Robins |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Kevin Feige |http://www.wikidata.org/prop/direct/P162 |producer |
| The Avengers |Joss Whedon |http://www.wikidata.org/prop/direct/P57 |director |
| The Avengers |Zak Penn |http://www.wikidata.org/prop/direct/P58 |screenwriter |
| The Avengers |Joss Whedon |http://www.wikidata.org/prop/direct/P58 |screenwriter |
| The Avengers |Alan Silvestri |http://www.wikidata.org/prop/direct/P86 |composer |
| The Avengers |Jeremy Renner |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Scarlett Johansson |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Gwyneth Paltrow |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Mark Ruffalo |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Jerzy Skolimowski |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Chris Hemsworth |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Clark Gregg |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Robert Downey Jr. |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Samuel L. Jackson |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Chris Evans |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Stan Lee |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Cobie Smulders |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Stellan Skarsgård |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Jenny Agutter |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Ashley Johnson |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Tom Hiddleston |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Harry Dean Stanton |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Alexis Denisof |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Powers Boothe |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |James Eckhouse |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Enver Gjokaj |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Kenneth Tigar |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Warren Kole |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Robert Clohessy |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Romy Rosemont |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Damion Poitier |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Jesse Garcia |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Thomas Roberts |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Walter Perez |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Maximiliano Hernández |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Alicia Sixtos |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Robin Swoboda |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Pat Kiernan |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Arthur Darbinyan |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Jillian Morgese |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Jamie McShane |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Brent McGee |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Josh Cowdery |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Donald Li |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Tina Benko |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |M'laah Kaur Singh |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Dieter Riesle |http://www.wikidata.org/prop/direct/P161 |cast member |
| The Avengers |Kirill Nikiforov |http://www.wikidata.org/prop/direct/P161 |cast member |


'showing 65 results'

Die so abgerufene Liste an Schauspielern bzw. Mitwirkenden dieses Films wird nun dem Datensatz der lokalen Datenbank hinzugefügt. Dazu werden die Daten zunächst aufbereitet und dann in dem Datensatz gespeichert.

In [1]:
from data.sparql import query, data, table, entity_to_json
import data.movies as movies


result = query(
"""
SELECT ?titleLabel ?castLabel ?property ?propLabel
WHERE {
    ?title wdt:P345 "tt0371746".
    # take all claims on this movie
    ?title ?property ?cast .
    # that involve a human
    ?cast wdt:P31 wd:Q5 .

    # get the property label
    # see https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries#Adding_labels_for_properties
    hint:Query hint:optimizer "None" .
    ?prop wikibase:directClaim ?property .

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
)
table(result)
dict = {"cast_members": []}
for cast_member in result["results"]["bindings"]:
    actor = {
        "Name": cast_member["castLabel"]["value"],
        "Role": cast_member["propLabel"]["value"]
    }
    dict["cast_members"].append(actor)


    #for key,value in cast_member.items():
    #    actor [key] = value["value"]
#print (dict)



movies.add_data_by_title("Iron Man", dict)
movies.get_by_title('Iron Man')

| titleLabel |castLabel |property |propLabel |
| --- | --- | --- | --- |
| Iron Man |Jon Favreau |http://www.wikidata.org/prop/direct/P57 |director |
| Iron Man |Mark Fergus |http://www.wikidata.org/prop/direct/P58 |screenwriter |
| Iron Man |Hawk Ostby |http://www.wikidata.org/prop/direct/P58 |screenwriter |
| Iron Man |Art Marcum |http://www.wikidata.org/prop/direct/P58 |screenwriter |
| Iron Man |Matt Holloway |http://www.wikidata.org/prop/direct/P58 |screenwriter |
| Iron Man |Ramin Djawadi |http://www.wikidata.org/prop/direct/P86 |composer |
| Iron Man |Gwyneth Paltrow |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Clark Gregg |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Shaun Toub |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Robert Downey Jr. |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Samuel L. Jackson |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Jeff Bridges |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Stan Lee |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Leslie Bibb |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Paul Bettany |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Jon Favreau |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Terrence Howard |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Tom Morello |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Faran Tahir |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Joshua Harto |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Tim Guinee |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Bill Smitrovich |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Nazanin Boniadi |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Meera Simhan |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Sayed Badreya |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Matthew Libatique |http://www.wikidata.org/prop/direct/P344 |director of photography |
| Iron Man |Dan Lebental |http://www.wikidata.org/prop/direct/P1040 |film editor |
| Iron Man |Stan Lee |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| Iron Man |Jon Favreau |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| Iron Man |Peter Billingsley |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| Iron Man |David Maisel |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| Iron Man |Louis D'Esposito |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| Iron Man |Ari Arad |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| Iron Man |Laura Jean Shannon |http://www.wikidata.org/prop/direct/P2515 |costume designer |
| Iron Man |Rebecca Bentjen |http://www.wikidata.org/prop/direct/P2515 |costume designer |
| Iron Man |J. Michael Riva |http://www.wikidata.org/prop/direct/P2554 |production designer |
| Iron Man |Will Lyman |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |America Olivo |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Peter Billingsley |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Adam Harrington |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Ahmed Ahmed |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Jim Cramer |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Lauren Scyphers |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Marco Khan |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Mike Cochrane |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Ricki Noel Lander |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Gabrielle Tuite |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Tim Griffin |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |James Bethea |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Sarah Cahill |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Fahim Fazli |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Russell Richardson |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Micah Hauptman |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Masha Lund |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Patrick O'Connell |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Daston Kalili |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Ido Mor |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Kevin Foster |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Garret Noël |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Eileen Weisinger |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Lana Kinnear |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Nicole Lindeblad |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Zorianna Kit |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Ben Newmark |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Jeannine Kaspar |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Stacy Stas Hurst |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Gerard Sanders |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Tim Rigby |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Thomas Craig Plumer |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Robert Berkman |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Frank Nyi |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Marvin Jordan |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Donna Evans |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Reid Harper |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Ava Rose Williams |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Summer Kylie Remington |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Vladimir Kubr |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Callie Croughwell |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Javan Tahir |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Sahar Bibiyan |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Justin Rex |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Kevin Feige |http://www.wikidata.org/prop/direct/P162 |producer |
| Iron Man |Avi Arad |http://www.wikidata.org/prop/direct/P162 |producer |
| Iron Man |Lauren Scyphers |http://www.wikidata.org/prop/direct/P175 |performer |
| Iron Man |Stacy Stas Hurst |http://www.wikidata.org/prop/direct/P175 |performer |


{'cast_members': [{'Name': 'Jon Favreau', 'Role': 'director'}, {'Name': 'Mark Fergus', 'Role': 'screenwriter'}, {'Name': 'Hawk Ostby', 'Role': 'screenwriter'}, {'Name': 'Art Marcum', 'Role': 'screenwriter'}, {'Name': 'Matt Holloway', 'Role': 'screenwriter'}, {'Name': 'Ramin Djawadi', 'Role': 'composer'}, {'Name': 'Gwyneth Paltrow', 'Role': 'cast member'}, {'Name': 'Clark Gregg', 'Role': 'cast member'}, {'Name': 'Shaun Toub', 'Role': 'cast member'}, {'Name': 'Robert Downey Jr.', 'Role': 'cast member'}, {'Name': 'Samuel L. Jackson', 'Role': 'cast member'}, {'Name': 'Jeff Bridges', 'Role': 'cast member'}, {'Name': 'Stan Lee', 'Role': 'cast member'}, {'Name': 'Leslie Bibb', 'Role': 'cast member'}, {'Name': 'Paul Bettany', 'Role': 'cast member'}, {'Name': 'Jon Favreau', 'Role': 'cast member'}, {'Name': 'Terrence Howard', 'Role': 'cast member'}, {'Name': 'Tom Morello', 'Role': 'cast member'}, {'Name': 'Faran Tahir', 'Role': 'cast member'}, {'Name': 'Joshua Harto', 'Role': 'cast member'}, {'N

{'index': 12588,
 'adult': 'False',
 'belongs_to_collection': "{'id': 131292, 'name': 'Iron Man Collection', 'poster_path': '/fbeJ7f0aD4A112Bc1tnpzyn82xO.jpg', 'backdrop_path': '/rI8zOWkRQJdlAyQ6WJOSlYK6JxZ.jpg'}",
 'budget': '140000000',
 'genres': "[{'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 12, 'name': 'Adventure'}]",
 'homepage': 'http://www.ironmanmovie.com/',
 'id': '1726',
 'imdb_id': 'tt0371746',
 'original_language': 'en',
 'original_title': 'Iron Man',
 'overview': 'After being held captive in an Afghan cave, billionaire engineer Tony Stark creates a unique weaponized suit of armor to fight evil.',
 'popularity': '22.073099',
 'poster_path': '/848chlIWVT41VtAAgyh9bWymAYb.jpg',
 'production_companies': "[{'name': 'Paramount Pictures', 'id': 4}, {'name': 'Marvel Studios', 'id': 420}]",
 'production_countries': "[{'iso_3166_1': 'US', 'name': 'United States of America'}]",
 'release_date': '2008-04-30',
 'revenue': 585174222.0,
 'runtime': 126.0,

Dieser Vorgang kann nun automatisiert für alle Einträge der lokalen Datenbank durchlaufen werden. 

In [12]:
from data.sparql import query, data, table, entity_to_json
import json
import data.movies as movies

def wikidata_query (imdb_id):
    return query(
    f"""
    SELECT ?titleLabel ?castLabel ?property ?propLabel
    WHERE {{
        ?title wdt:P345 "{imdb_id}".
        # take all claims on this movie
        ?title ?property ?cast .
        # that involve a human
        ?cast wdt:P31 wd:Q5 .

        # get the property label
        # see https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries#Adding_labels_for_properties
        hint:Query hint:optimizer "None" .
        ?prop wikibase:directClaim ?property .

        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    )

def extract_cast (wikidata_result):
    dict = {"cast_members": []}
    for cast_member in result["results"]["bindings"]:
        actor = {
            "Name": cast_member["castLabel"]["value"],
            "Role": cast_member["propLabel"]["value"]
        }
        dict["cast_members"].append(actor)
    return dict


#load all movies from local db (limited to 5):
local_movies = movies.get_all(10000)
for movie in local_movies:
    imdb_id = movie["imdb_id"]
    title = movie["original_title"]
    movie_id = movie["id"]
 
    #Query an 
    result = wikidata_query(imdb_id)
    
    cast_dict = extract_cast(result)
    
    movies.add_data_by_title(title, cast_dict)
    #print(json.dumps(movies.get_by_title(title), indent=1))
    
    print (f"handled movie: '{title}' with imdb {imdb_id}")

    

    



handled movie: 'Toy Story' with imdb tt0114709
handled movie: 'Jumanji' with imdb tt0113497
handled movie: 'Grumpier Old Men' with imdb tt0113228
handled movie: 'Waiting to Exhale' with imdb tt0114885
handled movie: 'Father of the Bride Part II' with imdb tt0113041
handled movie: 'Heat' with imdb tt0113277
handled movie: 'Sabrina' with imdb tt0114319
handled movie: 'Tom and Huck' with imdb tt0112302
handled movie: 'Sudden Death' with imdb tt0114576
handled movie: 'GoldenEye' with imdb tt0113189
handled movie: 'The American President' with imdb tt0112346
handled movie: 'Dracula: Dead and Loving It' with imdb tt0112896
handled movie: 'Balto' with imdb tt0112453
handled movie: 'Nixon' with imdb tt0113987
handled movie: 'Cutthroat Island' with imdb tt0112760
handled movie: 'Casino' with imdb tt0112641
handled movie: 'Sense and Sensibility' with imdb tt0114388
handled movie: 'Four Rooms' with imdb tt0113101
handled movie: 'Ace Ventura: When Nature Calls' with imdb tt0112281
handled movie: '

In [15]:
import json
import data.movies as movies

print(json.dumps(movies.get_by_title("Dingo"), indent=1))




{
 "index": 829,
 "adult": "False",
 "belongs_to_collection": NaN,
 "budget": "200000",
 "genres": "[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'name': 'Music'}]",
 "homepage": NaN,
 "id": "66634",
 "imdb_id": "tt0104109",
 "original_language": "en",
 "original_title": "Dingo",
 "overview": "Young John Anderson is captivated by jazz musician, Billy Cross when he performs on the remote airstrip of his Western Australian outback hometown after his plane is diverted. Years later, now a family man and making a meagre living tracking dingoes and playing trumpet in a local band, John still dreams of joining Billy on trumpet and makes a pilgrimage to Paris.",
 "popularity": "0.124273",
 "poster_path": "/aLPAqR2AckHSiS1v6JElHjeoY9V.jpg",
 "production_companies": "[{'name': 'Vertigo Productions', 'id': 2756}]",
 "production_countries": "[{'iso_3166_1': 'AU', 'name': 'Australia'}, {'iso_3166_1': 'FR', 'name': 'France'}]",
 "release_date": "1991-06-02",
 "revenue": 300000.0,
 "runtime": 109.0,
 "

In [ ]:
Frage: Wie kann man hier die Performance erhöhen

Zu welchen problemen kann es kommen und wie kann auf diese reagiert werden?

Frage:
Welche Daten kann man nicht einfach über Wikidata beziehen?
Bsp. 
- Wie lange liefen Filme in einem Kino?
- In Welchen Kinos lief ein Film?